[<img src='https://vermontcomplexsystems.org/index_files/large-526602.png' width="180" height="150" align="center"/>](https://vermontcomplexsystems.org/)


**CSYS 300:** Complex Networks

**Date:** 20230228

**Name:** Patrick L. Harvey

**Report:** Available on [Overleaf](https://www.overleaf.com/read/jwptqkrxjbkr)

****

**Index**

1. [Hedonometer API](##Hedonometer_API)

  a. [Citations and References](###Citation)

  b. [API](###API)

2. [Assignment 20](#Assignment_20)

  i. [Imports and Functions](##Imports_and_Functions)

  a. Part [1](##Question_1)

  b. Part [2](##Question_2)

  c. Part [3](##Question_3)

****

**Description:** This notebook applies [methods](https://arxiv.org/abs/2008.02250) developed by members of the Vermont Complex Systems Center to examine sentiment (e.g. happiness) for a given corpus.

## Hedonometer API

### Citations

```
@article{10.1371/journal.pone.0026752,
    doi = {10.1371/journal.pone.0026752},
    author = {
      Dodds, Peter Sheridan 
      AND Harris, Kameron Decker 
      AND Kloumann, Isabel M. 
      AND Bliss, Catherine A. 
      AND Danforth, Christopher M.
    },
    journal = {PLOS ONE},
    publisher = {Public Library of Science},
    title = {
      Temporal Patterns of Happiness and Information in a Global Social Network: Hedonometrics and Twitter
    },
    year = {2011},
    month = {12},
    volume = {6},
    url = {https://doi.org/10.1371/journal.pone.0026752},
    pages = {1-1},
    abstract = {
      Individual happiness is a fundamental societal metric.
      Normally measured through self-report, happiness has often 
      been indirectly characterized and overshadowed by more 
      readily quantifiable economic indicators such as gross 
      domestic product. Here, we examine expressions made on the 
      online, global microblog and social networking service 
      Twitter, uncovering and explaining temporal variations in 
      happiness and information levels over timescales ranging 
      from hours to years. Our data set comprises over 46 billion 
      words contained in nearly 4.6 billion expressions posted 
      over a 33 month span by over 63 million unique users. 
      In measuring happiness, we construct a tunable, real-time, 
      remote-sensing, and non-invasive, text-based hedonometer. 
      In building our metric, made available with this paper, 
      we conducted a survey to obtain happiness evaluations of 
      over 10,000 individual words, representing a tenfold size 
      improvement over similar existing word sets. Rather than 
      being ad hoc, our word list is chosen solely by frequency 
      of usage, and we show how a highly robust and tunable 
      metric can be constructed and defended.
    },
    number = {12}
}

@Misc{dodds2014a,
  author = {
    Dodds, P. S.
    and Clark, E. M.
    and Desu, S.
    and Frank, M. R.
    and Reagan, A. J.
    and  Williams, J. R.
    and Mitchell, L.
    and Harris, K. D.
    and Kloumann, I. M.
    and Bagrow, J. P.
    and Megerdoomian, K.
    and McMahon, M. T.
    and Tivnan, B. F.
    and Danforth, C. M.
  },
  title = {
    Human language reveals a universal positivity bias
  },
  OPThowpublished = {},
  OPTmonth = 	 {},
  year = {
    2014
  },
  note = 
  {
    Preprint available at
    \href{http://arxiv.org/abs/1406.3855}{http://arxiv.org/abs/1406.3855}
  },
  OPTannote = 	 {}
}
```












### API

In [ ]:
"""
API Links:
https://hedonometer.org/api/v1/words/?format=json&wordlist__title=labMT-en-v2
"""

import requests
import pandas as pd
import json

# TODO: Functionize

fmt   = 'json'
date  = '2018-01-01'
limit = '10000'
base  = 'http://hedonometer.org/api/v1/events/?format={}&'.format(fmt)
uri   = '{}happs__timeseries__title=en_all&'.format(base)
uri   = '{}happs__date__gte={}&limit={}'.format(uri, date, limit)
req   = requests.get(uri)
happiness_df = pd.DataFrame(json.loads(req.content)['objects'])

# TODO: Breakdown into sub DataFrames

for i in range(happiness_df.shape[0]):
  date = happiness_df['happs'][i]['date']
  frequency = happiness_df['happs'][i]['frequency']
  happiness = happiness_df['happs'][i]['happiness']
  timeseries = happiness_df['happs'][i]['timeseries']

# Assignment 20

## Imports and Functions

Load imports:

In [ ]:
%pip install -r requirements.txt --upgrade

import math
import matplotlib
import matplotlib.pyplot as plt
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import pandas as pd
import plotly
import seaborn as sns
import string
from string import punctuation as punct
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

: 

Define Functions:

In [3]:
def mount_drive():
  from google.colab import drive
  drive.mount('/content/drive/', force_remount = True)

def load_text(f: str = '/content/text.txt'):
  with open(f) as txt:
    in_str = txt.readlines()
  less_punct = ''.join(char for char in in_str if char not in set(punct))
  return

load_text(f = 'Blood_Meridian_McCarthy.txt')

# sentences = ["VADER is smart, handsome, and funny.", # positive sentence example
#     "VADER is smart, handsome, and funny!", # punctuation emphasis handled correctly (sentiment intensity adjusted)
#     "VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
#     "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
#     "VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
#     "VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!",# booster words & punctuation make this close to ceiling for score
#     "The book was good.",         # positive sentence
#     "The book was kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
#     "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
#     "A really bad, horrible book.",       # negative sentence with booster words
#     "At least it isn't a horrible book.", # negated negative sentence with contraction
#     ":) and :D",     # emoticons handled
#     "",              # an empty string is correctly handled
#     "Today sux",     #  negative slang handled
#     "Today sux!",    #  negative slang with punctuation emphasis handled
#     "Today SUX!",    #  negative slang with capitalization emphasis
#     "Today kinda sux! But I'll get by, lol" # mixed sentiment example with slang and constrastive conjunction "but"
#  ]
# paragraph = "It was one of the worst movies I've seen, despite good reviews. \
#  Unbelievably bad acting!! Poor direction. VERY poor production. \
#  The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie!"

# lines_list = tokenize.sent_tokenize(paragraph)
# sentences.extend(lines_list)
# tricky_sentences = [
#     "Most automated sentiment analysis tools are shit.",
#     "VADER sentiment analysis is the shit.",
#     "Sentiment analysis has never been good.",
#     "Sentiment analysis with VADER has never been this good.",
#     "Warren Beatty has never been so entertaining.",
#     "I won't say that the movie is astounding and I wouldn't claim that \
#     the movie is too banal either.",
#     "I like to hate Michael Bay films, but I couldn't fault this one",
#     "I like to hate Michael Bay films, BUT I couldn't help but fault this one",
#     "It's one thing to watch an Uwe Boll film, but another thing entirely \
#     to pay for it",
#     "The movie was too good",
#     "This movie was actually neither that funny, nor super witty.",
#     "This movie doesn't care about cleverness, wit or any other kind of \
#     intelligent humor.",
#     "Those who find ugly meanings in beautiful things are corrupt without \
#     being charming.",
#     "There are slow and repetitive parts, BUT it has just enough spice to \
#     keep it interesting.",
#     "The script is not fantastic, but the acting is decent and the cinematography \
#     is EXCELLENT!",
#     "Roger Dodger is one of the most compelling variations on this theme.",
#     "Roger Dodger is one of the least compelling variations on this theme.",
#     "Roger Dodger is at least compelling as a variation on the theme.",
#     "they fall in love with the product",
#     "but then it breaks",
#     "usually around the time the 90 day warranty expires",
#     "the twin towers collapsed today",
#     "However, Mr. Carter solemnly argues, his client carried out the kidnapping \
#     under orders and in the ''least offensive way possible.''"
#  ]
# sentences.extend(tricky_sentences)
# for sentence in sentences:
#      sid = SentimentIntensityAnalyzer()
#      print(sentence)
#      ss = sid.polarity_scores(sentence)
#      for k in sorted(ss):
#          print('{0}: {1}, '.format(k, ss[k]), end='')

NameError: name 'punct' is not defined

## Question 1

Using the main text you chose at the start of the semester, plot a time-series of happiness as described below using the labMT lexicon definitions of sentiment.

The labMT lexicon is referenced in ([Dodds](https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0026752&type=printable)) originally published in 2011. 

The lexicon has been occasionally upgraded to accommodate major changes in language use.
****

**Notes:**

• The horizontal axis is “narrative time” corresponding to 1-grams in the text, running from $1 \to N$.

• The windows should overlap, sliding one word ahead each time This is a simple averaging filter.

• Points should be located above the center of each window.

• So the point for the window running from $n$ to $n+T −1$ (T words) will be located at $n + (T − 1)/2$.

• Do not pre-filter the text for any given lens. Windows will contain variable numbers of words with and without happiness scores.
****

**Part a.**

Process (or destroy) your text so that it is a simple text file with one 1-gram per line—a vector of 1-grams.

To the extent possible, keep punctuation in as separate 1-grams. Periods, commas, semicolons, em-dashes, ellipses,
****

In [3]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


****
**Part b.**

First, use the full lexical lens provided by labMT.

Make a single figure containing a stacked set of 7 plots with text windows of
sizes:

$T = [10^z]$ for $z = 1, 1.5, 2, 2.5, 3, 3.5,$ and $4.0$.

Stacked here means separated and stacked vertically, as opposed to directly
overlaid. 

See examples for Moby Dick at the end of this assignment. 

The notation [$\cdot$] implying that we round to the nearest integer.
*****

****
**Part c.** 

Repeat the above for lenses which exclude the central words around the neutral point.

The blocked words are $h_{avg} \pm \delta h_{avg}$ where $\delta h_{avg} = 0.5, 1.0, 1.5, 2.0, 2.5, 3.0,$ and $3.5$.
****